# Building Delta Lake-house for US Immigirations
### Data Engineering Capstone Project

#### Project Summary
Building a data lake-house using [Delta Lake](https://delta.io/)</br>
Delta Lake is an open source storage layer that brings reliability to data lakes. Delta Lake provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing. Delta Lake runs on top of existing data lake and is fully compatible with Apache Spark APIs for more information follow [Delta Lake ducemntations](https://docs.delta.io/latest/index.html)</br>
The Delta architecture design pattern consists of landing data in successively cleaner Delta Lake tables from raw (Bronze) to clean (Silver) to aggregate (Gold). This is also referred to as a “multi-hop” architecture. You move data through this pipeline as needed.  
The end outputs are actionable insights, clean data for machine learning applications,  dashboards, and reports of business metrics.

![Architecture](images/delta_archticture.png)

The project follows the follow steps:
* [Step 1: Scope the Project and Gather Data](#step1)
* [Step 2: Explore and Assess the Data](#step2)
* [Step 3: Define the Data Model](#step3)
* [Step 4: Run ETL to Model the Data](#step4)
* [Step 5: Complete Project Write Up](#step5)

In [1]:
# This cell imports all nessecary libraries and configure spark and makes it available
%run ./utilities/config

importing libraries ....

Libraries (
    os, pandas as pd, glob, psutils, logging, findspark,
    pyspark.sql.SparkSession, delta.configure_spark_with_delta_pip,
    HTML, ImmigrationRaw, create_stream_writer, read_stream_delta,
    read_stream_raw, stop_all_streams, stop_named_stream, 
    until_stream_is_ready, register_delta_table, transform_raw
)
Are available now

pandas maximum column width is set to 400 and maximum number of columns to None
Setting up variables ....
vars_df is available as HTML content to display simply run HTML(vars_df)
Setting up spark configurations.....
spark session is now available in the environment as spark
spark_config_df is available as HTML content to display simply run HTML(spark_config_df)
Using immigration_db database..


In [2]:
# Display the variables availble in the enviroment
HTML(vars_df)

,Name,Value,Description
0,working_dir,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND,string path for current working directory
1,data_source,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\data_source,string path for data source location
2,delta_lake,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\delta_lake,string path for delta lake location
3,immigration_raw_path,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\delta_lake\immigrations_raw,string path for immigration raw data
4,immigration_bronz_path,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\delta_lake\immigration_bronz,string path for immigration bronz data
5,immigration_silver_path,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\delta_lake\immigration_silver,string path for immigration silver data
6,immigration_gold_path,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\delta_lake\immigration_gold,string path for immigration gold data
7,checkpoints_path,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\checkpoints,string path for checkpoints directory
8,immigration_bronz_checkpoint,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\checkpoints\immigration_bronz,string path for immigration bronz checkpoint
9,immigration_silver_checkpoint,D:\1- Study\In-progress\DEND\building-data-lake-house-DEND\checkpoints\immigration_silver,string path for immigration silver checkpoint


In [3]:
# Display Spark Configurations
HTML(spark_config_df)

,Config,Value,Description
0,spark.sql.extensions,io.delta.sql.DeltaSparkSessionExtension,Using delta io extension
1,spark.sql.catalog.spark_catalog,org.apache.spark.sql.delta.catalog.DeltaCatalog,Setting spark catalog to use DeltaCatalog
2,spark.ui.port,4050,Spark UI port number
3,spark.sql.shuffle.partitions,12,setting the number of shuffle partitions to the number of cores available
4,spark.sql.adaptive.enabled,True,Enabling adaptive query optimization
5,spark.memory.offHeap.enabled,True,Enabling offHeap memory
6,spark.memory.offHeap.size,10737418240,Setting offHeap memory to 10 GB


### Step 1: Scope the Project and Gather Data <a id='step1'></a>

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [4]:
immigration_raw = ImmigrationRaw(immigration_path, immigration_raw_path)

In [5]:
immigration_raw.retrive()

Retriving... i94_jan16_sub part: 12: 100%|█████████████████████████████████████████████| 12/12 [00:51<00:00,  4.33s/it]


In [6]:
immigration_raw.retrived

{'i94_jan16_sub': 2847924}

In [7]:
immigration_raw.source

['D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_feb16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_mar16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_apr16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_may16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_jun16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_jul16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i94_aug16_sub.sas7bdat',
 'D:\\1- Study\\In-progress\\DEND\\building-data-lake-house-DEND\\data_source\\18-83510-I94-Data-2016\\i

In [8]:
immigration_raw.schema

'cicid DOUBLE, i94yr DOUBLE, i94mon DOUBLE, i94cit DOUBLE, i94res DOUBLE, i94port STRING, arrdate DOUBLE, i94mode DOUBLE, i94addr STRING, depdate DOUBLE, i94bir DOUBLE, i94visa DOUBLE, count DOUBLE, dtadfile STRING, visapost STRING, occup STRING, entdepa STRING, entdepd STRING, entdepu STRING, matflag STRING, biryear DOUBLE, dtaddto STRING, gender STRING, insnum STRING, airline STRING, admnum DOUBLE, fltno STRING, visatype STRING'

In [9]:
immigrations_stream_df = read_stream_raw(spark, immigration_raw_path, schema=immigration_raw.schema)

In [10]:
immigrations_transformed_df = transform_raw(immigrations_stream_df, 'gharib_local_pc', 'i94mon')

In [11]:
immigrations_raw_writer = create_stream_writer(immigrations_transformed_df,
                                               immigration_bronz_checkpoint,
                                               'immigrations_raw_to_bronz',
                                               'p_i94mon')

In [12]:
immigrations_raw_to_bronz = immigrations_raw_writer.start(immigration_bronz_path)

In [13]:
immigrations_raw_to_bronz.isActive

True

In [14]:
immigrations_raw_to_bronz.status

{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [15]:
until_stream_is_ready(spark, 'immigrations_raw_to_bronz')


The stream immigrations_raw_to_bronz is active and ready.


True

In [16]:
register_delta_table(spark, 'immigration_bronz', immigration_bronz_path)

In [4]:
delta_imm = spark.read.table('immigration_bronz')

In [5]:
print(f'{delta_imm.count():,}')

40,790,529


In [5]:
delta_imm.limit(10).toPandas().to_csv('s3://gharib/databricks/')

ImportError: Install s3fs to access S3

In [19]:
print(f'{immigration_raw.rows_retrived:,}')

2,847,924


In [21]:
assert delta_imm.count() == immigration_raw.rows_retrived, "number of rows retrived doesn't match"
print("Assertion passed.")

Assertion passed.


In [34]:
immigration_raw.retrive(20)

Retriving... i94_dec16_sub part: 14: 100%|█████████████████████████████████████████████| 14/14 [01:40<00:00,  7.19s/it]


In [35]:
immigrations_raw_to_bronz.status

{'message': 'Getting offsets from FileStreamSource[file:/D:/1- Study/In-progress/DEND/building-data-lake-house-DEND/delta_lake/immigrations_raw]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [36]:
print(f'{delta_imm.count():,}')

40,607,539


In [37]:
immigrations_raw_to_bronz.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [38]:
print(f'{delta_imm.count():,}')

40,607,539


In [39]:
print(f'{immigration_raw.rows_retrived:,}')

40,790,529


In [40]:
delta_imm.count() - immigration_raw.rows_retrived

0

In [41]:
spark.sql("SELECT COUNT(1) FROM immigration_bronz").toPandas().iloc[0, 0]

40790529

In [42]:
spark.sql("SELECT * FROM immigration_bronz LIMIT 5").toPandas()

,cicid,i94yr,p_i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,data_source,ingest_time,ingest_date
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346608285.0,424,F1,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346627585.0,424,F1,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,17.0,2.0,1.0,None,None,None,T,N,None,M,1999.0,07152016,F,None,AF,381092385.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,45.0,2.0,1.0,None,None,None,T,N,None,M,1971.0,07152016,F,None,AF,381087885.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,12.0,2.0,1.0,None,None,None,T,N,None,M,2004.0,07152016,M,None,AF,381078685.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22


In [53]:
spark.sql("""
    SELECT 
        p_i94mon,
        format_number(COUNT(1), 0) AS Count
    FROM 
        immigration_bronz 
    GROUP BY 
        p_i94mon 
    ORDER BY
        p_i94mon
""").toPandas()

,p_i94mon,Count
0,1.0,"2,847,924"
1,2.0,"2,570,543"
2,3.0,"3,157,072"
3,4.0,"3,096,313"
4,5.0,"3,444,249"
5,6.0,"3,574,989"
6,7.0,"4,265,031"
7,8.0,"4,103,570"
8,9.0,"3,733,786"
9,10.0,"3,649,136"


In [44]:
assert spark.sql("SELECT COUNT(1) FROM immigration_bronz").toPandas().iloc[0, 0] == immigration_raw.rows_retrived, "number of rows retrived doesn't match"
print("Assertion passed.")

Assertion passed.


In [45]:
immigration_raw.retrive(9)

In [46]:
print(f'{delta_imm.count():,}')

40,790,529


In [47]:
immigration_raw.source

[]

In [48]:
delt = spark.sql("SELECT format_number(COUNT(1), 0) AS count FROM immigration_bronz").toPandas()

In [49]:
delt

,count
0,"40,790,529"


In [50]:
for stream in spark.streams.active:
    print(stream.name)
#     stream.stop()

immigrations_raw_to_bronz


In [51]:
spark.sql("SELECT * FROM immigration_bronz LIMIT 10").toPandas()

,cicid,i94yr,p_i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,data_source,ingest_time,ingest_date
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346608285.0,424,F1,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,None,None,None,T,None,None,None,1996.0,D/S,M,None,LH,346627585.0,424,F1,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,17.0,2.0,1.0,None,None,None,T,N,None,M,1999.0,07152016,F,None,AF,381092385.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,45.0,2.0,1.0,None,None,None,T,N,None,M,1971.0,07152016,F,None,AF,381087885.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,12.0,2.0,1.0,None,None,None,T,N,None,M,2004.0,07152016,M,None,AF,381078685.0,338,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
5,12.0,2016.0,1.0,101.0,101.0,BOS,20474.0,1.0,MA,NaN,33.0,2.0,1.0,None,None,None,T,None,None,None,1983.0,07202016,M,None,LH,406155985.0,424,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
6,15.0,2016.0,1.0,101.0,101.0,BOS,20477.0,1.0,MA,20524.0,28.0,3.0,1.0,None,None,None,T,O,None,M,1988.0,D/S,F,None,LH,417363085.0,424,F1,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
7,17.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,78.0,2.0,1.0,None,None,None,T,None,None,None,1938.0,07262016,M,None,TK,428558285.0,81,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
8,18.0,2016.0,1.0,101.0,101.0,BOS,20480.0,1.0,MA,NaN,70.0,2.0,1.0,None,None,None,T,None,None,None,1946.0,07262016,F,None,TK,428561085.0,81,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22
9,20.0,2016.0,1.0,101.0,101.0,CHI,20473.0,1.0,IL,20482.0,28.0,2.0,1.0,None,None,None,T,O,None,M,1988.0,07192016,M,None,BA,401779785.0,295,B2,gharib_local_pc,2021-06-22 12:31:15.222,2021-06-22


In [52]:
spark.sql("DESCRIBE HISTORY immigration_bronz").toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,157,2021-06-22 12:59:32.665,None,None,STREAMING UPDATE,"{'epochId': '157', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,156.0,None,True,"{'numOutputRows': '182990', 'numRemovedFiles': '0', 'numAddedFiles': '6', 'numOutputBytes': '3864290'}",None
1,156,2021-06-22 12:59:16.684,None,None,STREAMING UPDATE,"{'epochId': '156', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,155.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '5402138'}",None
2,155,2021-06-22 12:59:08.926,None,None,STREAMING UPDATE,"{'epochId': '155', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,154.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '5375715'}",None
3,154,2021-06-22 12:59:01.440,None,None,STREAMING UPDATE,"{'epochId': '154', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,153.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '5357028'}",None
4,153,2021-06-22 12:58:54.020,None,None,STREAMING UPDATE,"{'epochId': '153', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,152.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '5323347'}",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,4,2021-06-22 12:33:35.740,None,None,STREAMING UPDATE,"{'epochId': '4', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,3.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '3818347'}",None
154,3,2021-06-22 12:33:29.484,None,None,STREAMING UPDATE,"{'epochId': '3', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,2.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '3930104'}",None
155,2,2021-06-22 12:33:23.087,None,None,STREAMING UPDATE,"{'epochId': '2', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,1.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '9', 'numOutputBytes': '3846587'}",None
156,1,2021-06-22 12:33:17.281,None,None,STREAMING UPDATE,"{'epochId': '1', 'outputMode': 'Append', 'queryId': 'c519d304-3f4a-4ba4-a267-922c4821dd4b'}",None,None,None,0.0,None,True,"{'numOutputRows': '250000', 'numRemovedFiles': '0', 'numAddedFiles': '8', 'numOutputBytes': '4080997'}",None


In [56]:
# Read in the data here
sample_i94 = pd.read_csv(immigration_sample_path, index_col=0)

In [57]:
sample_i94.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


### Step 2: Explore and Assess the Data <a id='step2'></a>
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model <a id='step3'></a>
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data <a id='step4'></a>
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up <a id='step5'></a>
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.